## Nezapomínat "Clear all Outputs" a kdykoliv něco načítá nebo exportuje, připojit před to "data/" !

Tahle verze papá vstupní tabulky "05_shifts_to_boats_04.csv" a "01_cruises.csv" a kaká harbour - situaci v přístavu hodinu po hodině 

Některé úpravy jsou pevně vázány na čísla indexů, tak bacha když budem chtít něco vepisovat

In [ ]:
import pandas as pd
from datetime import timedelta

In [ ]:
cruises = pd.read_csv('data/01_cruises.csv')
cruises = cruises[['Dato','ETA','ETD','Navn','Passasjerer','Kai', 'Agent','Tonn', 'Lengde']]
cruises = cruises.rename(columns={"Dato": "date", "Navn": "boat", "Passasjerer": "passengers", "Kai": "quay", "Tonn": "weight", "Lengde": "lenght", 'Agent': 'agent'})
cruises.head(20)

In [ ]:
# updating times for World Traveler, Jewel of the Sea, getting rid of Queen Victoria and Hamburg
def corrections(df, updates):
    for row, collumn, value in updates:
        if pd.isna(df.loc[row, collumn]):
            df.loc[row, collumn] = value

# Define all updates in a list of tuples
update = [
    (16, 'ETA', '08:00'),
    (16, 'ETD', '17:00'),
    (49, 'ETA', '14:00'), 
    (113, 'ETD', '23:00')
]

corrections(cruises, update)


In [ ]:
# Tady pracovně vyhazuju i Trollfjorda na indexu 160 (17.11.2023) - zjistili jsme, že nepřijel
cruises = cruises.drop(index=[15, 47, 115, 160], axis=0) 


In [ ]:
replacements = {
    'Aidabella': 'AIDAbella',
    'Aidaluna': 'AIDAluna',
    'AidaMar': 'AIDAmar',
    'Aidasol': 'AIDAsol',
    'Preziosa': 'MSC Preziosa',
    'Sverdrup': 'Otto Sverdrup',
    'Sverdrup ?': 'Otto Sverdrup',
    'Hamburg ?': 'Hamburg',
    'Ocean Majesty?': 'Ocean Majesty',
    'Mein Schiff': 'Mein Schiff 4',
    'Island Princess?': 'Island Princess',
    'Nieuw Statendam?': 'Nieuw Statendam',
    "Borealis 61'": 'Borealis',
    'Europa (Skarsv?g)': 'Europa',
    'Pacific World (p1800)': 'Pacific World',
    'Queen Viktoria (2200)': 'Queen Viktoria',
    'Ambience p1000': 'Ambience',
    'MS Marina p600': 'MS Marina',
    'Riviera p1258': 'Riviera',
    'Preziosa ': 'MSC Preziosa',
    'Preziosa': 'MSC Preziosa',
    'Ambition?': 'Ambition',
    'Greg Mortimer p160': 'Greg Mortimer', 
    'Sevens Seas Navigator': 'Seven Seas Navigator', 
    'Seabourn Oviation': 'Seabourn Ovation'
}

cruises['boat'] = cruises['boat'].replace(replacements)

cruises

In [ ]:
cruises[["Day", "Month", "Year"]] = cruises["date"].str.split(".", expand=True)
cruises["Date of arrival"] = cruises["Year"] + "-" + cruises["Month"]+ "-" + cruises["Day"]
cruises["Date of departure"] = cruises ["Date of arrival"]
cruises = cruises.drop(["date", "Day", "Month", "Year"], axis="columns")
cruises = cruises[['Date of arrival', 'Date of departure', 'ETA', 'ETD', 'boat', 'passengers', 'quay', 'agent',
       'weight', 'lenght']]
cruises

In [ ]:
cruises["Date"] = cruises["Date of arrival"]
cruises.to_csv("data/cruises.csv", encoding="utf-8")

## Tabulka Shifts (obsahuje přesnější ETA/ETD lodí):

In [ ]:
shifts = pd.read_csv("data/05_shifts_to_boats_04.csv")
shifts

In [ ]:
shifts[["Month", "Day", "Year"]] = shifts["date"].str.split("/", expand=True)
shifts["Month"] = shifts["Month"].str.rjust(2, "0")
shifts["Day"] = shifts["Day"].str.rjust(2, "0")
shifts["Date"] = shifts["Year"] + "-" + shifts["Month"] + "-" + shifts["Day"]

shifts = shifts.drop(["Month","Day","Year", "date"], axis="columns")
shifts

In [ ]:
shifts["time"] = shifts["time"].str.strip()

shifts[["ETA", "ETD"]] = shifts["time"].str.split("-", expand=True) 
shifts["ETA"] = shifts["ETA"].str.strip()
shifts["ETA"] = shifts["ETA"].str.strip(":")
shifts["ETA"] = shifts["ETA"].str.strip("?")
shifts["ETD"] = shifts["ETD"].str.strip()
shifts["ETD"] = shifts["ETD"].str.strip(":")
shifts["ETD"] = shifts["ETD"].str.strip("?")



In [ ]:
def corrections(df, updates):
    for row, collumn, value in updates:
        df.loc[row, collumn] = value
            

# Define all updates in a list of tuples
update = [
    (116, 'ETA', '1400'),
    (116, 'ETD', '2000'),
]

# Apply all updates in one function call
corrections(shifts, update)
# shifts.to_csv("shifts.csv", encoding="utf-8")
shifts['ETD'] = shifts['ETD'].replace({"2400":"2359"})



In [ ]:
prvni_pulka_ETA = shifts['ETA'].str.slice(0, 2)
druha_pulka_ETA = shifts['ETA'].str.slice(2, 5)
shifts["ETA"] = prvni_pulka_ETA + ":" + druha_pulka_ETA
prvni_pulka_ETD = shifts['ETD'].str.slice(0, 2)
druha_pulka_ETD = shifts['ETD'].str.slice(2, 5)
shifts["ETD"] = prvni_pulka_ETD + ":" + druha_pulka_ETD

shifts.loc[shifts['ETA'] == ":", 'ETA'] = ""
shifts.loc[shifts['ETD'] == ":", 'ETD'] = ""
## casy vyresene, ted nazvy
shifts


In [ ]:
replacements = {
    'Aidabella': 'AIDAbella',
    'Aidaluna': 'AIDAluna',
    'AidaMar': 'AIDAmar',
    'Aidasol': 'AIDAsol',
    'Preziosa': 'MSC Preziosa',
    'Sverdrup': 'Otto Sverdrup',
    'Sverdrup ?': 'Otto Sverdrup',
    'Hamburg ?': 'Hamburg',
    'Ocean Majesty?': 'Ocean Majesty',
    'Mein Schiff': 'Mein Schiff 4',
    'Island Princess?': 'Island Princess',
    'Nieuw Statendam?': 'Nieuw Statendam',
    "Borealis 61'": 'Borealis',
    'Europa (Skarsv?g)': 'Europa',
    'Pacific World (p1800)': 'Pacific World',
    'Queen Viktoria (2200)': 'Queen Viktoria',
    'Ambience p1000': 'Ambience',
    'MS Marina p600': 'MS Marina',
    'Riviera p1258': 'Riviera',
    'Preziosa ': 'MSC Preziosa',
    'Preziosa': 'MSC Preziosa',
    'Ambition?': 'Ambition',
    'Greg Mortimer p160': 'Greg Mortimer',
    'Vasgo Da Gama': 'Vasco Da Gama',
    'Arcadia p2388': 'Arcadia', 
    'Jewel of the seas': 'Jewel of the Seas'
}
shifts["boat"] = shifts["boat"].str.strip(" ?")
shifts['boat'] = shifts['boat'].replace(replacements)

shifts

In [ ]:
shifts.to_csv("data/shifts.csv")

## Spojení tabulek, ať můžem nahradit časy z cruises časy z shifts

In [ ]:
combined = pd.merge(cruises, shifts, on=["Date", "boat"], how="outer")


In [ ]:
# Pokud je nějaký čas uvedený v tabulce shifts (a není to prázdný řetězec nebo NaN --> nahraď čas v tabulce cruises tímto časem  

combined['ETA_x'] = combined['ETA_x'].mask(combined['ETA_y'].notna() & (combined['ETA_y'] != ''), combined['ETA_y'])
combined['ETD_x'] = combined['ETD_x'].mask(combined['ETD_y'].notna() & (combined['ETD_y'] != ''), combined['ETD_y'])



In [ ]:
# Zbav se zbytečných sloupečků, pokud možno všech, co jsou původně tabulka shifts


combined = combined.drop(['Date', 'time', 'ETA_y','ETD_y'], axis="columns")

combined.head(20)


In [ ]:
combined["ETA date"] = combined["Date of arrival"] + " " + combined["ETA_x"] + ":00"
combined["ETD date"] = combined["Date of arrival"] + " " + combined["ETD_x"] + ":00"

# # Změní čas na typ datetime místo objectu, co to byl původně 
combined["ETA date"] = pd.to_datetime(combined["ETA date"], errors='coerce')
combined["ETD date"] = pd.to_datetime(combined["ETD date"], errors='coerce')

# # Vypočítá rozdíl v časech - vyhazuje to - 1 den když loď přijede před půlnocí a odjede na druhý den
combined["Time in harbour"] = combined["ETD date"] - combined["ETA date"]

combined['Date of arrival'] = pd.to_datetime(combined['Date of arrival'])
combined['Date of departure'] = pd.to_datetime(combined['Date of departure'])
combined.info()


In [ ]:
# Definice podmínky, za které se přičte den navíc do sloupečku "Date of departure"
podminka_pridani_dne = combined["Time in harbour"] < "0 days 00:00:00"

# Aplikace podmínky
combined.loc[podminka_pridani_dne, "Date of departure"] = combined["Date of departure"] + timedelta(days=1)
combined.loc[podminka_pridani_dne, "ETD date"] = combined["ETD date"] + timedelta(days=1)
# manuálně přidávám den k Jewel of the sea, protože to jinak neumím
combined.loc[47, "Date of departure"] += timedelta(days=1)
combined.loc[47, "ETD date"] += timedelta(days=1)
#Vyhodím druhý Jewel of theseas, co se tam dostala asi z Shifts při mergování
combined = combined.drop(index=45, axis=0) 


In [ ]:
combined.info()

In [ ]:
combined.columns

### Dataframe "combined" v tuhle chvíli obsahuje sjednocená jména lodí a časy opravené podle tabulky shifts

In [ ]:


# Function to generate hourly data
def expand_rows(row):
    hours = pd.date_range(row['ETA date'], row['ETD date'], freq='h')
    return pd.DataFrame({
        'hourly_timestamp': hours,
        # 'ETA date': row['ETA date'],
        # 'ETD date': row['ETD date'],
        # 'Unnamed: 0': row['Unnamed: 0'], 
        'Date of arrival': row['Date of arrival'],
        'Date of departure': row['Date of departure'], 
        'ETA_x': row['ETA_x'], 
        'ETD_x': row['ETD_x'], 
        'boat': row['boat'],
        'passengers': row['passengers'], 
        'quay': row['quay'],
        'agent': row['agent'], 
        'weight': row['weight'], 
        'lenght': row['lenght'],
        # 'Time in harbour': row['Time in harbour'],  
    })

# Apply the function and concatenate the results
combined = pd.concat([expand_rows(row) for index, row in combined.iterrows()], ignore_index=True)

combined = combined.drop(['Date of arrival', 'Date of departure', 'ETA_x','ETD_x'], axis="columns")

combined.to_csv("data/harbour_for_reals.csv", encoding="utf-8")

### Holy Macaroni, popletla jsem Jewel of the Seas, opravim, ale potřebuju čas :D 
#### Jinak to podle mě fachčí